In [1]:
!git clone https://github.com/CaptainMaxine/A2A-MCP.git
%cd A2A-MCP


Cloning into 'A2A-MCP'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 119 (delta 40), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (119/119), 50.89 KiB | 1002.00 KiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/A2A-MCP


In [2]:
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.0/334.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.3 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 8.4.2
    Uninstalling pytest-8.4.2:
      Successfully uninstalled pytest-8.4.2


In [3]:
!python -m mcp_server.server


[MCP Server] Creating database at /content/A2A-MCP/mcp_server/support.db ...
Connected to database: /content/A2A-MCP/mcp_server/support.db
Tables created successfully!
Triggers created successfully!
Sample data inserted successfully!
  - 15 customers added
  - 25 tickets added
Database connection closed.
[MCP Server] Database initialized with sample data.
[MCP Server] Database stored at: /content/A2A-MCP/mcp_server/support.db
[MCP Server] Tools ready for MCPClient (in-process mode).


In [4]:
from agents.coordinator import A2ACoordinator

coordinator = A2ACoordinator()
queries = [
    "I need help with my account, customer ID 12345",
    "I want to cancel my subscription but I'm having billing issues",
    "What's the status of all high-priority tickets for premium customers?",
    "Get customer information for ID 5",
    "I'm customer 12345 and need help upgrading my account",
    "Show me all active customers who have open tickets",
    "I've been charged twice, please refund immediately!",
    "Update my email to new@email.com and show my ticket history",
]

for q in queries:
    print("====", q, "====")
    response, log = coordinator.run(q)
    for line in log:
        print(line)
    print("FINAL RESPONSE:", response)
    print("\n")


==== I need help with my account, customer ID 12345 ====
[STEP 1] user → router | content=I need help with my account, customer ID 12345 | state={}
[STEP 2] router → customer_data | content=[Router] Dispatching scenario=scenario_1_task_allocation to customer_data. | state={'original_query': 'I need help with my account, customer ID 12345', 'customer_id': 12345, 'intent': 'account_help', 'issue': 'help with your account', 'scenario': 'scenario_1_task_allocation', 'step': 1, 'action': 'get_customer'}
[STEP 3] customer_data → router | content=[CustomerDataAgent] No customer found for id=12345. | state={'original_query': 'I need help with my account, customer ID 12345', 'customer_id': 12345, 'intent': 'account_help', 'issue': 'help with your account', 'scenario': 'scenario_1_task_allocation', 'step': 1, 'action': 'get_customer', 'customer': None}
[STEP 4] router → support | content=[Router] Forwarding customer context to SupportAgent. | state={'original_query': 'I need help with my account